# 05: Shapley Values 

See sections in Molnar's book on [Shapley Values](https://christophm.github.io/interpretable-ml-book/shapley.html) and [SHAP](https://christophm.github.io/interpretable-ml-book/shap.html) for background information. For actual use, see the [shap package](https://github.com/slundberg/shap).

## Imports

In [ ]:
from dataclasses import dataclass, field
from itertools import product
import random

import altair as alt
import numpy as np
import pandas as pd
import pmlb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay

In [ ]:
# If you're running this code locally, then you can uncomment this to automatically
# save the chart data in files, rather than including the data in the spec. 

!mkdir -p data
alt.data_transformers.enable('json', prefix='data/altair-data')

## Data Preparation and Modeling

For this lab, we'll be using a bike rental dataset. This is a regression dataset where the goal is to predict the number of bikes that were rented at a particular day and time. This dataset is from the [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset). The data processing was guided by [Molnar's IML book](https://christophm.github.io/interpretable-ml-book/bike-data.html).

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/DanielKerrigan/f324b392dc9a58d8bd8f8d79e1101a12/raw/c3b4760c9facfac26bcab2cd7465c4cab88ef304/bike-hour.csv')

To reduce computation times, we'll drop some of the columns.

In [ ]:
df.drop(columns=['yr', 'mnth', 'atemp'], inplace=True)

In [ ]:
df.head()

We'll use the data from 2011 for training.

In [ ]:
df_train = df[df['days_since_2011'] < 365].copy()

In [ ]:
X_train = df_train.drop(columns=['cnt'])
y_train = df_train['cnt'].values

Next we'll train a random forest model on this dataset. We'll do a grid search with cross-validation to find reasonable hyperparameters.

In [ ]:
param_grid = {
    'n_estimators': [10],
    'bootstrap': [True],
    'max_features': ['sqrt', 1.0],
    'max_depth': [6, 12],
    'min_samples_split': [2, 8],
}

cv = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1)

cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
cv.best_score_

In [ ]:
model = cv.best_estimator_

## Shapley Implementation

**Exercises 1:**

First, we will write a function to approximately calculate a feature's Shapley value for a given instance. Our algorithm will be similar to the one that Molnar details in [Section 9.5.3.3](https://christophm.github.io/interpretable-ml-book/shapley.html#estimating-the-shapley-value).

*1a)* Select a random instance from the dataframe `df`. [df.sample()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html#pandas-dataframe-sample) is useful for this.

*1b)* Select a random set of features, not including the feature that we are calculating the shapley value for (`feature`). [random.randrange()](https://docs.python.org/3/library/random.html#random.randrange) and [random.sample()](https://docs.python.org/3/library/random.html#random.sample) are useful for this.

*1c)* Make a copy of the instance `x`. For the features randomly selected in 1b, replace the value in `x` with the value in the random instance from 1a.

*1d)* Make a copy of the instance from 1c. Replace the value of `feature` with the value from the random instance from 1a.

*1e)* Get the predicted values of the instances from 1c and 1d. Calculate the difference between the predictions.

In [ ]:
'''
df - dataframe containing the entire dataset
x - dataframe containing a single instance
model - trained sklearn model
feature - the name of the feature that we are computing the Shapley value for
iterations - number of iterations to run for
'''
def calculate_shapley_value(df, x, model, feature, iterations):
    # keep track of the total from the summation
    value = 0
    
    # list of features besides the one we are computing the shapley value for
    other_features = [f for f in df.columns if f != feature]

    for _ in range(iterations):
        # 1a: get a random instance from the df
        random_instance = df.sample()
        
        # 1b: select a random set of features
        num_features_to_change = random.randrange(len(other_features))
        features_to_change = random.sample(other_features, num_features_to_change)
        
        # 1c: make a copy of the instance x for the randomly selected features,
        # replace the value of that feature in x with the value in random_instance
        z_original = x.copy()
        
        for f in features_to_change:
            z_original[f] = random_instance[f].values
            
        # 1d: make a copy of z_original. replace the value
        # of feature with the value in random_instance
        z_different = z_original.copy()
        z_different[feature] = random_instance[feature].values
        
        
        # 1e: get the predicted values for z_original and z_different.
        # calculate the difference between them
        pred_original = model.predict(z_original)[0]
        pred_different = model.predict(z_different)[0]
        difference = pred_original - pred_different
        
        value += difference
        
    # take the mean
    return value / iterations

In [ ]:
calculate_shapley_value(X_train, X_train.iloc[[0]], model, 'hr', 50)

The below `shapley_values` function calculates the shapley value of every feature for `num_instances` instances. It returns a dataframe containing the shapley values.

In [ ]:
def shapley_values(df, model, iterations):
    rows = []
    
    for i in range(df.shape[0]):
        x = df.iloc[[i]]
        
        row = {}
        
        for feature in df.columns:
            row[feature] = calculate_shapley_value(df, x, model, feature, iterations)
            
        rows.append(row)
        
    return pd.DataFrame(rows)

In [ ]:
subset = X_train.sample(100).reset_index(drop=True)

In [ ]:
shapley = shapley_values(subset, model, 50)

In [ ]:
shapley

## Visualizations

### Feature Importance Bar Chart

**Exercise 2:** Create a bar chart that shows the feature importance of each feature based on the shapley values.

*2a)* Calculate the mean absolute values for each feature in `shapley`. The [mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html) and [abs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.abs.html) functions will be useful. We will want a dataframe that has two columns: one for the feature and one for the value.

In [ ]:
feature_importance = pd.DataFrame(shapley.abs().max()).reset_index()
feature_importance.columns = ['feature', 'value']

feature_importance

*2b)* Plot the feature importances in a bar chart.

In [ ]:
alt.Chart(feature_importance).mark_bar().encode(
    y=alt.Y('feature', sort='-x'),
    x=alt.X('value', title='mean absolute shapley value')
)

### Dependence Scatter Plot

**Exercise 3:** For a given feature, we can create a scatterplot that shows the relationship between an instance's value for that feature (x-axis) and its shapley value for that feature (y-axis). This works as an alternative to PDPs. Complete the function below to create a dependence plot for the given feature.

*3a)* Create a dataframe containing the feature values and shapley values for the given `feature`. This dataframe should have two columns: feature_value and shapley_value. Each row represents an instance.

*3b)* Return a scatterplot of the dataframe.

In [ ]:
def plot_dependence(instances, shapley, feature):
    # 3a) create a dataframe containing the feature values and shapley values 
    dependence = pd.DataFrame({
        'feature_value': instances[feature],
        'shapley_value': shapley[feature]
    })
    
    # 3b) plot the values in a scatterplot
    return alt.Chart(dependence).mark_point().encode(
        x=alt.X('feature_value', title=feature),
        y='shapley_value'
    )

In [ ]:
plot_dependence(subset, shapley, 'hr')

In [ ]:
plot_dependence(subset, shapley, 'temp')

### Summary Strip Plot